In [2]:
# Modified from https://github.com/ClementPerroud/RL-Trading-Agent/blob/main/manage_data.ipynb

import pandas as pd
import numpy as np
import glob
from pathlib import Path
from tqdm.auto import tqdm  

from gym_trading_env.downloader import download
import datetime

In [3]:
download_dir = "data/raw"
processed_dir = "data/processed/"

TARGET_TIMEFRAME = pd.Timedelta("30m")
download_timeframe = "5m"

In [6]:
download(
    exchange_names = [
        # "binance",
        "bitfinex2",
        "huobi"
    ],
    symbols= ["BTC/USDT", "ETH/USDT"],
    timeframe= download_timeframe,
    dir = download_dir,
    since= datetime.datetime(year= 2019, month= 1, day=1),
)

BTC/USDT downloaded from huobi and stored at data/raw/huobi-BTCUSDT-5m.pkl
BTC/USDT downloaded from bitfinex2 and stored at data/raw/bitfinex2-BTCUSDT-5m.pkl
ETH/USDT downloaded from huobi and stored at data/raw/huobi-ETHUSDT-5m.pkl
ETH/USDT downloaded from bitfinex2 and stored at data/raw/bitfinex2-ETHUSDT-5m.pkl


In [5]:
pathes = glob.glob(f"{download_dir}/*pkl")
for path in tqdm(pathes):
    name = Path(path).name.split(".")[0]
    df  = pd.read_pickle(path)
    timeframe = (df.index - df.index.to_series().shift(1)).value_counts().index[0]
    for offset in range(TARGET_TIMEFRAME//timeframe):
        process_df = df.resample("30min", offset= offset*timeframe).agg({
            "date_close": lambda x : x.iloc[-1] if len(x) > 0 else np.nan,
            "open": lambda x : x.iloc[0] if len(x) > 0 else np.nan,
            "high": lambda x : max(x) if len(x) > 0 else np.nan,
            "low": lambda x : min(x) if len(x) > 0 else np.nan,
            "close": lambda x : x.iloc[-1] if len(x) > 0 else np.nan,
            "volume": lambda x : sum(x) if len(x) > 0 else np.nan
        })[1:-1]
        process_df.dropna(inplace = True)
        # process_df.to_pickle(f"data/processed/{name}-{offset}.pkl")
        bp = int(0.9 * process_df.shape[0]);
        process_df.iloc[bp:, :].to_pickle(f"{processed_dir}/validation/{name}-{offset}.pkl");
        process_df.iloc[:bp, :].to_pickle(f"{processed_dir}/training/{name}-{offset}.pkl")

  0%|          | 0/4 [00:00<?, ?it/s]